### Download Flowers dataset

In [65]:
# #You can download the data manually as well instead of using 'wget'
# !wget http://download.tensorflow.org/example_images/flower_photos.tgz --quiet

In [66]:
#Check if file is downloaded
!ls -l

total 643176
-rw-rw-r--@ 1 vidyadharbendre  staff      32125 Mar 22 20:05 1a. Classification_MNIST_CNN_Keras.ipynb
-rw-r--r--@ 1 vidyadharbendre  staff      80277 Mar 22 22:09 1b. Classification_MNIST_CNN_Keras_Functional.ipynb
-rw-rw-r--@ 1 vidyadharbendre  staff    1269188 Mar 22 21:07 2a. Visualize an Image.ipynb
-rw-------@ 1 vidyadharbendre  staff      49116 Mar 23 07:54 2b. Image_Classification_CNN.ipynb
-rw-rw-r--@ 1 vidyadharbendre  staff      41563 Mar 23 10:13 2c. Image Classification - Flowers.ipynb
drwxr-xr-x@ 5 vidyadharbendre  staff        160 Mar 22 20:54 check_point_folder
drwxr-x---@ 8 vidyadharbendre  staff        256 Feb 11  2016 flower_photos
-rw-r--r--@ 1 vidyadharbendre  staff  228813984 Feb 11  2016 flower_photos.tgz
-rw-r--r--@ 1 vidyadharbendre  staff   77390704 Mar 23 10:13 flowers.h5
-rw-r--r--@ 1 vidyadharbendre  staff    4842280 Mar 22 22:06 mnist_functional_template_v3.h5
drwxr-xr-x@ 6 vidyadharbendre  staff        192 Mar 22 20:54 save_mode_folder


In [67]:
# #Unzip the data
# !tar -xf flower_photos.tgz

In [68]:
#Check how data is organized
!ls -l flower_photos

total 824
-rw-r-----@   1 vidyadharbendre  staff  418049 Feb  9  2016 LICENSE.txt
drwx------@ 635 vidyadharbendre  staff   20320 Feb 11  2016 daisy
drwx------@ 900 vidyadharbendre  staff   28800 Feb 11  2016 dandelion
drwx------@ 643 vidyadharbendre  staff   20576 Feb 11  2016 roses
drwx------@ 701 vidyadharbendre  staff   22432 Feb 11  2016 sunflowers
drwx------@ 801 vidyadharbendre  staff   25632 Feb 11  2016 tulips


### Build batch generator

In [69]:
# Import the NumPy library, commonly aliased as 'np', for numerical computations and array operations.
import numpy as np

# Import the Pandas library, commonly aliased as 'pd', for data manipulation and analysis.
import pandas as pd

# Import the Matplotlib library, specifically the pyplot module, for data visualization.
import matplotlib.pyplot as plt

In [70]:
# Import the TensorFlow library, commonly aliased as 'tf', for building and training neural networks.
import tensorflow as tf

# Import the MNIST dataset from the TensorFlow Keras datasets module.
from tensorflow.keras.datasets import mnist

# Import the to_categorical function from the TensorFlow Keras utils module.
# This function is used for one-hot encoding of categorical labels.
from tensorflow.keras.utils import to_categorical

# Import the clear_session function from the TensorFlow Keras backend module.
# This function is used to clear the Keras session and reset the TensorFlow graph.
from tensorflow.keras.backend import clear_session

# Import necessary libraries and functions from TensorFlow.

# Import the Sequential and Model classes from the TensorFlow Keras models module.
# These classes are used to create sequential and functional API models, respectively.
from tensorflow.keras.models import Sequential, Model, load_model

# Import various layer classes (Reshape, Dense, BatchNormalization, LeakyReLU, Input)
# from the TensorFlow Keras layers module.
# These classes are used to define the architecture of neural network models.
from tensorflow.keras.layers import Input, Reshape, Flatten, Dense, Conv2D, MaxPool2D, BatchNormalization, Dropout, LeakyReLU

# Import callback classes (ModelCheckpoint, EarlyStopping) from the TensorFlow Keras callbacks module.
# These classes are used to define callbacks for monitoring and controlling the training process.
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [71]:
#Define some parameters
img_size = 60
img_depth = 3

Create an ImageDataGenerator object, it can also split data between train and test.

In [72]:
#ImageDataGenerator declaration with 20% data as test (80% for training)
img_generator= tf.keras.preprocessing.image.ImageDataGenerator(validation_split=0.2)

ImageDataGenerator can read images directory and also resize them if needed

In [73]:
#Build training generator. 
train_generator = img_generator.flow_from_directory('flower_photos',
                                                    batch_size=64,
                                                    target_size=(img_size, img_size),
                                                    subset='training')

Found 2939 images belonging to 5 classes.


In [74]:
#Build test generator
test_generator = img_generator.flow_from_directory('flower_photos',
                                                   target_size=(img_size, img_size),                                                   
                                                   subset='validation',
                                                   batch_size=64)

Found 731 images belonging to 5 classes.


In [75]:
type(train_generator)

keras.preprocessing.image.DirectoryIterator

ImageDataGenerator returns 64 images and their labels

In [76]:
#Lets check the features (images) and Labels (flower class) returned by ImageDataGenerator
X, y = next(train_generator)

In [77]:
print('Input features shape', X.shape)
print('Actual labels shape', y.shape)

Input features shape (64, 60, 60, 3)
Actual labels shape (64, 5)


In [78]:
y[0]

array([0., 0., 0., 1., 0.], dtype=float32)

In [79]:
import numpy as np
np.unique(X[0])

array([  0.,   1.,   2.,   3.,   4.,   5.,   6.,   7.,   8.,   9.,  10.,
        11.,  12.,  13.,  14.,  15.,  16.,  17.,  18.,  19.,  20.,  21.,
        22.,  23.,  24.,  25.,  26.,  27.,  28.,  29.,  30.,  31.,  32.,
        33.,  34.,  35.,  36.,  37.,  38.,  39.,  40.,  41.,  42.,  43.,
        44.,  45.,  46.,  47.,  48.,  49.,  50.,  51.,  52.,  53.,  54.,
        55.,  56.,  57.,  58.,  59.,  60.,  61.,  62.,  63.,  64.,  65.,
        66.,  67.,  68.,  69.,  70.,  71.,  72.,  73.,  74.,  75.,  76.,
        77.,  78.,  79.,  80.,  81.,  82.,  83.,  84.,  85.,  86.,  87.,
        88.,  89.,  90.,  91.,  92.,  93.,  94.,  95.,  96.,  97.,  98.,
        99., 100., 101., 102., 103., 104., 105., 106., 107., 108., 109.,
       110., 111., 112., 113., 114., 115., 116., 117., 118., 119., 120.,
       121., 122., 123., 124., 125., 126., 127., 128., 129., 130., 131.,
       132., 133., 134., 135., 136., 137., 138., 139., 140., 141., 142.,
       143., 144., 145., 146., 147., 148., 149., 15

In [80]:
train_generator.class_indices

{'daisy': 0, 'dandelion': 1, 'roses': 2, 'sunflowers': 3, 'tulips': 4}

In [81]:
X[0]

array([[[111., 141., 167.],
        [116., 137., 166.],
        [116., 137., 166.],
        ...,
        [113., 128., 157.],
        [113., 128., 157.],
        [114., 129., 158.]],

       [[117., 136., 166.],
        [118., 139., 168.],
        [117., 141., 169.],
        ...,
        [116., 131., 160.],
        [118., 132., 161.],
        [116., 130., 159.]],

       [[118., 139., 168.],
        [120., 141., 170.],
        [120., 141., 170.],
        ...,
        [116., 131., 160.],
        [117., 133., 159.],
        [114., 131., 157.]],

       ...,

       [[ 20.,  30.,   5.],
        [ 33.,  36.,  15.],
        [ 68.,  56.,   4.],
        ...,
        [ 36.,  33.,  16.],
        [ 35.,  36.,  18.],
        [ 31.,  34.,  13.]],

       [[ 65.,  42.,   0.],
        [ 74.,  59.,   4.],
        [ 58.,  45.,  10.],
        ...,
        [ 43.,  45.,  24.],
        [ 80.,  87.,  45.],
        [ 20.,  22.,   9.]],

       [[ 68.,  45.,  11.],
        [ 66.,  56.,  21.],
        [ 29.,  

### Build CNN Model

In [82]:
# #Clear any previous model from memory
# tf.keras.backend.clear_session()

# #Initialize model
# model = tf.keras.models.Sequential()

# #normalize data
# model.add(tf.keras.layers.BatchNormalization(input_shape=(img_size,img_size,3,)))

# #Add Conv Layer
# model.add(tf.keras.layers.Conv2D(32, kernel_size=(3,3), activation='relu'))

# #normalize data
# model.add(tf.keras.layers.BatchNormalization())

# #Add Conv Layer
# model.add(tf.keras.layers.Conv2D(64, kernel_size=(3,3), activation='relu'))

# #normalize data
# model.add(tf.keras.layers.BatchNormalization())

# #Add Max Pool layer
# model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2)))

# #Add Dense Layers after flattening the data
# model.add(tf.keras.layers.Flatten())
# model.add(tf.keras.layers.Dense(128, activation='relu'))

# #Add Dropout
# model.add(tf.keras.layers.Dropout(0.25))

# #Add Output Layer
# model.add(tf.keras.layers.Dense(5, activation='softmax'))

In [84]:
### version -3 ###
# Clear any existing TensorFlow graph/session.
clear_session()

# Define the input layer with a shape of (28, 28, 1), representing 28x28 images with 1 channel.
inputs = Input(shape=(img_size, img_size, 3))
input_batch_norm_1 = BatchNormalization()(inputs)
# Add a Conv2D layer with 32 filters, a kernel size of 3x3, ReLU activation function, and batch normalization.
conv_layer_1 = Conv2D(32, kernel_size=(3, 3), activation="relu", strides=(1,1), kernel_initializer='he_normal', name="conv_layer_1")(input_batch_norm_1)
conv_batch_norm_2 = BatchNormalization()(conv_layer_1)

# Add a Conv2D layer with 64 filters, a kernel size of 3x3, ReLU activation function, and batch normalization.
conv_layer_2 = Conv2D(64, kernel_size=(3, 3), activation="relu", name="conv_layer_2")(conv_batch_norm_2)
conv_batch_norm_2 = BatchNormalization()(conv_layer_2)

# Add MaxPooling layer
max_pooling_1 = MaxPool2D(pool_size=(2,2), strides=2)(conv_batch_norm_2)

# Flatten the output of the convolutional layer.
flattened = Flatten()(max_pooling_1)

# Define the architecture of the neural network using fully connected (Dense) layers.
# Reshape data from 2D to 1D -> 28x28 to 784

# Add the first hidden layer with 128 neurons, ReLU activation function, and batch normalization.
first_hidden_layer = Dense(128, activation="relu", name="first_hidden_layer")(flattened)
#first_batch_norm = BatchNormalization()(first_hidden_layer)
first_drop = Dropout(0.25)(first_hidden_layer)


# Add the output layer with 10 neurons (for 10 classes) and softmax activation function.
outputs = Dense(5, activation='softmax')(first_drop)

# Define the model with input and output layers.
model = Model(inputs=inputs, outputs=outputs)

In [85]:
# Compile the model with stochastic gradient descent (SGD) optimizer, categorical crossentropy loss,
# and accuracy metric for evaluation during training.
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [86]:
#Model Summary
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 60, 60, 3)]       0         
                                                                 
 batch_normalization (BatchN  (None, 60, 60, 3)        12        
 ormalization)                                                   
                                                                 
 conv_layer_1 (Conv2D)       (None, 58, 58, 32)        896       
                                                                 
 batch_normalization_1 (Batc  (None, 58, 58, 32)       128       
 hNormalization)                                                 
                                                                 
 conv_layer_2 (Conv2D)       (None, 56, 56, 64)        18496     
                                                                 
 batch_normalization_2 (Batc  (None, 56, 56, 64)       256   

In [87]:
#Model Summary
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 60, 60, 3)]       0         
                                                                 
 batch_normalization (BatchN  (None, 60, 60, 3)        12        
 ormalization)                                                   
                                                                 
 conv_layer_1 (Conv2D)       (None, 58, 58, 32)        896       
                                                                 
 batch_normalization_1 (Batc  (None, 58, 58, 32)       128       
 hNormalization)                                                 
                                                                 
 conv_layer_2 (Conv2D)       (None, 56, 56, 64)        18496     
                                                                 
 batch_normalization_2 (Batc  (None, 56, 56, 64)       256   

### Train the model

In the context of machine learning, both model checkpoints and callbacks are tools used to improve the training process of neural networks, particularly in deep learning frameworks like TensorFlow and PyTorch. However, they serve different purposes:

Model Checkpoint:

A model checkpoint is a feature used to save the model's current weights or entire state at certain intervals during training.
The primary purpose of a model checkpoint is to save the model's progress periodically so that it can be restored or used later for inference or further training.
Model checkpoints are typically used to save the best-performing model based on a specified metric, such as validation accuracy or loss. This ensures that you retain the best version of the model throughout the training process.
Checkpoints are especially useful for long training processes where interruptions or failures may occur, allowing you to resume training from the last saved checkpoint rather than starting from scratch.
Callback:

A callback is a function or set of functions that are executed at specific points during the training process, such as at the beginning or end of an epoch or after a batch of data is processed.
Callbacks are more versatile and can perform various tasks during training, such as logging training metrics, adjusting learning rates dynamically, implementing early stopping, or saving model checkpoints.
While model checkpoints are a specific type of callback used for saving model weights, callbacks can perform a wide range of additional functions to customize the training process according to your specific needs.
Callbacks can be defined by the user or provided by the deep learning framework as pre-defined functions. Users can also create custom callbacks to implement specific behaviors not covered by built-in callbacks.
In summary, a model checkpoint is a type of callback used specifically for saving model weights or state during training, while callbacks encompass a broader range of functions that can be executed at various stages of the training process to customize and enhance the training procedure.

In [88]:
# ModelCheckpoint callback to save the best model based on validation accuracy
model_checkpoint = ModelCheckpoint(
    'flowers.h5',  # File path to save the model
    save_best_only=True,       # Save only the best model based on monitored metric
    monitor='val_accuracy',    # Metric to monitor for determining the best model
    mode='max',                # Mode for determining the best model ('max' or 'min')
    verbose=1                  # Verbosity mode (0 or 1)
)

# EarlyStopping callback to stop training when the monitored metric stops improving
early_stopping = EarlyStopping(
    monitor='val_loss',        # Metric to monitor for early stopping
    patience=5,                # Number of epochs with no improvement after which training will be stopped
    verbose=1,                 # Verbosity mode (0 or 1)
    restore_best_weights=True  # Whether to restore model weights from the epoch with the best value of the monitored quantity
)

In [89]:
2939//64

45

In [90]:
# model.fit(train_generator,
#           epochs=200,
#           steps_per_epoch= 2939//64,  #Number of batches per epoch
#           validation_data=test_generator,
#           validation_steps = 731//64, 
#           callbacks=[model_checkpoint]) #Number of test images//batch_size

In [91]:
# Train the model on the training data (X_train and y_train) with the following configurations:
# - Validation data: (X_test, y_test), used to evaluate the model after each epoch.
# - Number of epochs: 2, specifying how many times the entire training dataset is passed through the model.
# - Batch size: 32, indicating the number of samples per gradient update during training.
# - Callbacks: ModelCheckpoint and EarlyStopping, used for saving the best model and stopping early to prevent overfitting.
model.fit(train_generator,
          epochs=2,
          steps_per_epoch= 2939//64,  #Number of batches per epoch          
          validation_data=test_generator,
          validation_steps = 731//64,
          callbacks=[model_checkpoint, early_stopping])

Epoch 1/2


2024-03-23 10:13:59.731571: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


45/45 [==============================] - ETA: 0s - loss: 4.9682 - accuracy: 0.3558

2024-03-23 10:14:02.454355: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.



Epoch 1: val_accuracy improved from -inf to 0.28267, saving model to flowers.h5
45/45 [==============================] - 4s 72ms/step - loss: 4.9682 - accuracy: 0.3558 - val_loss: 1.5671 - val_accuracy: 0.2827
Epoch 2/2
45/45 [==============================] - ETA: 0s - loss: 1.4507 - accuracy: 0.4240
Epoch 2: val_accuracy improved from 0.28267 to 0.33523, saving model to flowers.h5
45/45 [==============================] - 3s 68ms/step - loss: 1.4507 - accuracy: 0.4240 - val_loss: 1.4678 - val_accuracy: 0.3352


In [62]:
#initial_epoch=5,  # Start training from the 3th epoch (indexing starts from 0)

In [92]:
model.fit(train_generator,
          epochs=10,
          steps_per_epoch= 2939//64,  #Number of batches per epoch          
          validation_data=test_generator,
          validation_steps = 731//64,
          initial_epoch=2,
          callbacks=[model_checkpoint, early_stopping])

Epoch 3/10
45/45 [==============================] - ETA: 0s - loss: 1.2553 - accuracy: 0.4515
Epoch 3: val_accuracy improved from 0.33523 to 0.43324, saving model to flowers.h5
45/45 [==============================] - 3s 70ms/step - loss: 1.2553 - accuracy: 0.4515 - val_loss: 1.3864 - val_accuracy: 0.4332
Epoch 4/10
44/45 [============================>.] - ETA: 0s - loss: 1.1341 - accuracy: 0.5233
Epoch 4: val_accuracy improved from 0.43324 to 0.46449, saving model to flowers.h5
45/45 [==============================] - 3s 68ms/step - loss: 1.1321 - accuracy: 0.5249 - val_loss: 1.3564 - val_accuracy: 0.4645
Epoch 5/10
45/45 [==============================] - ETA: 0s - loss: 1.0343 - accuracy: 0.5704
Epoch 5: val_accuracy improved from 0.46449 to 0.49006, saving model to flowers.h5
45/45 [==============================] - 3s 68ms/step - loss: 1.0343 - accuracy: 0.5704 - val_loss: 1.3861 - val_accuracy: 0.4901
Epoch 6/10
45/45 [==============================] - ETA: 0s - loss: 0.9256 - ac

In [93]:
model.fit(train_generator,
          epochs=25,
          steps_per_epoch= 2939//64,  #Number of batches per epoch          
          validation_data=test_generator,
          validation_steps = 731//64,
          initial_epoch=10,
          callbacks=[model_checkpoint, early_stopping])

Epoch 11/25
45/45 [==============================] - ETA: 0s - loss: 0.6152 - accuracy: 0.7464
Epoch 11: val_accuracy did not improve from 0.55824
45/45 [==============================] - 3s 69ms/step - loss: 0.6152 - accuracy: 0.7464 - val_loss: 1.5430 - val_accuracy: 0.5582
Epoch 12/25
45/45 [==============================] - ETA: 0s - loss: 0.5859 - accuracy: 0.7652
Epoch 12: val_accuracy improved from 0.55824 to 0.56534, saving model to flowers.h5
45/45 [==============================] - 3s 71ms/step - loss: 0.5859 - accuracy: 0.7652 - val_loss: 1.5193 - val_accuracy: 0.5653
Epoch 13/25
45/45 [==============================] - ETA: 0s - loss: 0.5154 - accuracy: 0.7868
Epoch 13: val_accuracy improved from 0.56534 to 0.57102, saving model to flowers.h5
45/45 [==============================] - 3s 70ms/step - loss: 0.5154 - accuracy: 0.7868 - val_loss: 1.4632 - val_accuracy: 0.5710
Epoch 14/25
44/45 [============================>.] - ETA: 0s - loss: 0.5292 - accuracy: 0.8008
Epoch 14: 